In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import requests
from selenium.webdriver.common.keys import Keys

from datetime import datetime
import threading

In [42]:
driver = webdriver.Chrome('./chromedriver')
time.sleep(2)

driver.get('https://v4.map.naver.com/')
time.sleep(2)

# 팝업창 지우기
driver.find_element_by_xpath('//div[@id="dday_popup"]//div[2]//button').click()

word = input('검색하고싶은 지역을 입력하세요 : ')
s = driver.find_element_by_id('search-input')
s.send_keys(word)
s.send_keys(Keys.RETURN)
time.sleep(2)

page = 1
d = {}
while len(d) < 20:
    thread = threading.Thread(target = cafe(d))
    thread.start()
    thread.join()
    nextPage(page, word)
    time.sleep(2)
    page += 1
    
#print(d)

검색하고싶은 지역을 입력하세요 : 애월읍 카페
{'새빌': [4.03, '리뷰수: 250'], '봄날': [4.38, '리뷰수: 259'], '랜디스도넛 제주직영점': [4.23, '리뷰수: 323'], '초록달 과자점': [4.56, '리뷰수: 39'], '지금이순간카페': [4.44, '리뷰수: 233'], '애월더선셋': [4.02, '리뷰수: 109'], '너와의 첫여행': [4.62, '리뷰수: 57'], '인디안 썸머 애월': [4.88, '리뷰수: 20'], '마틸다': [4.43, '리뷰수: 38'], '썬셋클리프': [3.94, '리뷰수: 31'], '애월당 애월제과': [4.31, '리뷰수: 103'], '애월빵공장&카페': [4.04, '리뷰수: 35'], '리치망고 애월본점': [4.37, '리뷰수: 95'], '이너프': [4.31, '리뷰수: 32'], '카페브리프': [4.51, '리뷰수: 36'], '레이지펌프': [4.27, '리뷰수: 44'], '당당': [4.33, '리뷰수: 63'], '섬앤썸': [4.08, '리뷰수: 74'], '모립': [4.29, '리뷰수: 14'], '골목카페옥수': [4.5, '리뷰수: 17'], '정직한돈 본점': [4.63, '리뷰수: 79'], 'SALON de LAVANT': [4.64, '리뷰수: 11'], '버터모닝': [4.58, '리뷰수: 33'], '언니커피': [4.6, '리뷰수: 21']}


In [55]:
df = pd.DataFrame(d)

In [56]:
df.index = ['별점', '리뷰수']

In [57]:
df = df.T
df.head()

,별점,리뷰수
새빌,4.03,리뷰수: 250
봄날,4.38,리뷰수: 259
랜디스도넛 제주직영점,4.23,리뷰수: 323
초록달 과자점,4.56,리뷰수: 39
지금이순간카페,4.44,리뷰수: 233


In [58]:
df.리뷰수 = df.리뷰수.apply(lambda x : int(x.split(' ')[1]))

In [60]:
df.to_excel('애월읍카페_네이버.xlsx')

In [40]:
len(d)

26

In [41]:
def reviewCrawl(summ, count, month, page, nid):
    # 카페마다 영수증 리뷰보기
    url = 'https://store.naver.com/restaurants/detail?id={}&tab=receiptReview&tabPage={}'.format(nid, page)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    ul1 = soup.select('ul.list_receipt_review')
    lis1 = ul1[0].select('li')
    date = []
    for i in lis1:
        if month == '06':  # 월은 유동적으로 바꿀수있습니다
            c = '리뷰수: ' + str(count)
            return [round((summ / count), 2), c]
        s = float(i.select('span.score')[0].text) # 별점
        summ += s   # 별점의 합
        count += 1  # 리뷰의 수
        month = i.select('div.reviewer_area > div.reviewer > span')[0].text.split('.')[1]  # 리뷰가 작성된 월
    
    # Recursive function; 함수 안에서 같은 함수 부르기; 페이지 넘길때 쓰는용도
    return reviewCrawl(summ, count, month, page + 1, nid)

In [29]:
def cafe(d):
    html = driver.page_source
    soup = bs(html, 'html.parser')
    ul = soup.select('ul.lst_site')
    lis = ul[0].select('li.detail')

    for li in lis:
        try:
            summ = 0
            count = 0
            month = ''
            page = 1
            nid = li.select('a')[0]['data-id'][1:] # 카페의 고유 ID 뽑기
            url = 'https://store.naver.com/restaurants/detail?id={}'.format(nid) # 카페 상세페이지 들어가기
            driver.get(url)
            time.sleep(2)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            key = soup.select('strong.name')[0].text  # 카페 이름
            avg = reviewCrawl(summ, count, month, page, nid) # 별점 평균
            d[key] = avg  # 카페와 평균 별점을 dict에 저장
        except:
            continue # 리뷰가 없으면 건너뛰기

In [36]:
def nextPage(page, word):
    driver.get('https://v4.map.naver.com/')
    driver.find_element_by_xpath('//div[@id="dday_popup"]//div[2]//button').click()
    time.sleep(2)
    search(word)
    time.sleep(2)
    if page % 5 == 0:
        driver.find_element_by_link_text('다음').click()
    else:
        driver.find_element_by_link_text(str(page+1)).click()

In [32]:
def search(word):
    s = driver.find_element_by_id('search-input')
    s.send_keys(word)
    s.send_keys(Keys.RETURN)